In [ ]:
!wget https://www.openslr.org/resources/17/musan.tar.gz

In [ ]:
!tar -zxvf musan.tar.gz

In [ ]:
!pip install datasets

In [ ]:
# Load MUSAN dataset

from datasets import load_dataset, DatasetDict
noise_ds = load_dataset('./musan')

In [ ]:
# Resample MUSAN dataset to 44.1kHz
import torchaudio.transforms as T
def resample_audio(batch, new_freq):
    audio_array = torch.FloatTensor(batch['audio']['array'])
    original_sampling_rate = batch['audio']['sampling_rate']
    
    if original_sampling_rate != 16000:
        raise ValueError("Expected original sampling rate to be 16kHz.")
    
    resampler = T.Resample(orig_freq=16000, new_freq=new_freq)
    resampled_audio = resampler(audio_array).squeeze().numpy()
    
    batch['audio']['array'] = resampled_audio
    batch['audio']['sampling_rate'] = new_freq
    return batch

# Apply the resampling function to the dataset
noise_ds = noise_ds['train'].map(resample_audio, new_freq=44100)

In [ ]:
# load paired jar-alt dataset (train80 valid10 test10)
dataset_p = load_dataset('audioshake/jam-alt', revision='v1.0.0')
train_trainvalid_p = dataset_p['test'].train_test_split(test_size=0.2)

# Split the 10% test + valid in half test, half valid
test_valid_p = train_trainvalid_p['test'].train_test_split(test_size=0.5)
paired_ds = DatasetDict({
    'train': train_trainvalid_p['train'],
    'test': test_valid_p['test'],
    'valid': test_valid_p['train']})

In [ ]:
# add noise randomly to dataset
import numpy as np
import random
import torchaudio
import soundfile

def add_noise_to_audio(audio, noise, clean_db, noise_snr):
    noise_db = 10 * np.log10(np.mean(noise ** 2)+1e-4) 
    noises = [np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noise]

    if len(noises[0]) < len(audio):
        added = np.pad(noises[0], (0, len(audio)-len(noises[0])), 'constant') + audio
    else:
        added = audio + noises[0][:len(audio)]
 
    return added


def add_noise_to_dataset(batch, noise_dataset):#, noise_sample=None):
    noise_sample = noise_dataset[random.randint(0, len(noise_dataset) - 1)]
    noise_array = noise_sample['audio']['array'] * 30

    audio_array = batch['audio']['array']
    clean_db = 10 * np.log10(np.mean(audio_array ** 2)+1e-4) ##
    audio_array = audio_array.squeeze()#.numpy()
    
    noisy_audio = add_noise_to_audio(audio_array, noise_array, clean_db, noise_snr=10)
    
    batch["audio"] = noisy_audio

    return batch

In [ ]:
dataset = paired_ds.map(lambda batch: add_noise_to_dataset(batch, noise_ds['train']))